### Import Dump

In [1]:
import numpy as np
import pandas as pd
from fastai.tabular.all import *
from ydata_profiling import ProfileReport as pr
from sklearn.model_selection import train_test_split

c:\Users\paoma\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


KeyboardInterrupt: 

### Surface-Level Analysis

In [26]:
path = "retail_sales_dataset.csv"

# read dataset
df = pd.read_csv(path)

df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [ ]:
# create dataprofile for general overview
profile = pr(df)

profile.to_file("data_profile.html")

### Model

In [48]:
# split into train and test dataset
train_df = df[200:]
test_df = df[:200]

In [49]:
# create preprocessing table for creating normalized values
tp = TabularPandas(
    df = df,
    procs = [Categorify, FillMissing, Normalize],
    cat_names = ["Gender", "Product Category"],
    cont_names = ["Age"],
    y_names = "Total Amount",
    y_block  = RegressionBlock(),
    splits = RandomSplitter(valid_pct=0.2)(range_of(df))
)

tp.xs.iloc[:5]

,Gender,Product Category,Age
606,2,2,0.900855
970,1,3,-1.068426
65,1,3,0.244428
471,1,1,-0.266126
194,2,2,0.754982


In [50]:
dls = tp.dataloaders(bs=50)

dls.show_batch()

,Gender,Product Category,Age,Total Amount
0,Female,Electronics,40.000000,60.0
1,Female,Beauty,51.000000,600.0
2,Male,Electronics,46.000000,1000.0
3,Female,Clothing,64.000000,30.0
4,Male,Clothing,31.000000,200.0
5,Female,Beauty,56.999999,100.0
6,Female,Electronics,24.000000,50.0
7,Male,Clothing,18.000001,1200.0
8,Female,Clothing,22.000000,2000.0
9,Female,Electronics,25.000000,300.0


In [51]:
# create model
model = tabular_learner(dls, layers=[1000,500],
                        metrics=exp_rmspe, 
                        loss_func=MSELossFlat())

# train model
model.fit_one_cycle(5)

epoch,train_loss,valid_loss,_exp_rmspe,time
0,475279.937500,530487.187500,nan,00:00
1,486976.812500,529776.750000,nan,00:00
2,497184.437500,528805.437500,nan,00:00
3,496763.906250,528680.000000,nan,00:00
4,497425.406250,528206.500000,nan,00:00


In [23]:
model.show_results()

,Gender,Product Category,Age,Total Amount,Total Amount_pred
0,2.0,3.0,43.0,300.0,2.102037
1,1.0,3.0,48.0,50.0,0.946562
2,2.0,1.0,19.0,2000.0,1.863365
3,1.0,3.0,52.0,150.0,-0.326571
4,1.0,2.0,50.0,200.0,0.452634
5,2.0,2.0,59.0,25.0,-2.365858
6,1.0,1.0,57.0,200.0,-1.696722
7,1.0,2.0,27.0,2000.0,0.911816
8,2.0,1.0,39.0,25.0,2.065388
